In [9]:
!pip install datasets huggingface_hub
!pip install pyarrow --upgrade

In [10]:
!huggingface-cli login
#надо сначала создать свой access token в huggingface



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGr

In [11]:
#удаляет папку images
import shutil
shutil.rmtree("/content/images")

In [12]:
from datasets import load_dataset
old_urls = load_dataset("I77/house_kg_flats")["train"].to_pandas()["url"].unique().tolist()

README.md:   0%|          | 0.00/399 [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/634M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/492M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/473M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/477M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1696 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv
import os

link_base = "https://www.house.kg/kupit-kvartiru?page="

columns = ["price",
           "link"]

data = {i: [] for i in columns}

def get_text_for_analytic(text):
    res = ""
    flag = False
    for i in text:
        if i == "<":
            flag = True
        elif i == ">":
            flag = False
        elif flag:
            continue
        elif i not in ("\t", "\n"):
            res += i

    return res


left_border = int(input("From which page do you want to parse: "))
right_border = int(input("Until which page do you want to parse: "))
step_to_save = 3
counter = 0
counter_saves = 1
counter_skip = 0

print("#PARSING STARTED#")
flat_id = 0

for i in range(left_border, right_border):  # 640
    link = link_base + str(i)
    page_req = requests.get(link)
    page_soup = BeautifulSoup(page_req.text, "html.parser")
    page_table = page_soup.find("div", attrs={"class": "listings-wrapper"})
    if page_table == None:
        continue
    counter += 1
    flats = page_table.find_all("div", attrs={"itemtype": "https://schema.org/Apartment", "class": "listing"})
    for flat in flats:
        flat_link = "https://www.house.kg" + flat.find("a")["href"]
        if flat_link in data["link"] or flat_link in old_urls: #уберите вторую часть условия если у вас пока что не было датасета
          print(counter_skip+1)
          counter_skip += 1
          continue
        flat_data = {i: None for i in columns}
        flat_data["link"] = flat_link

        flat_req = requests.get(flat_link)
        flat_soup = BeautifulSoup(flat_req.text, "html.parser")

        header = flat_soup.find("div", attrs={"class": "details-header"})
        description_check = flat_soup.find("div", attrs={"class": "details-main"})

        price = header.find('div', attrs={"class": "price-som"})
        if price != None:
            flat_data["price"] = price.text.strip()

        save_dir = "images"
        os.makedirs(save_dir, exist_ok=True)

        right_side = description_check.find("div", attrs={"class": "right"})
        images = right_side.find_all("a")
        os.makedirs(save_dir+"/"+str(flat_id), exist_ok=True)

        for idx, photo_url in enumerate(images):
            try:

                if images[idx]["title"] != "Фото":
                    continue
                photo_url = images[idx]["href"]
                img_data = requests.get(photo_url).content
                with open(os.path.join("images/"+str(flat_id), f"{flat_id}_{idx}.jpg"), "wb") as f:
                    f.write(img_data)
            except:
                continue

        flat_id += 1
        for column in flat_data:
            data[column].append(flat_data[column])



From which page do you want to parse: 0
Until which page do you want to parse: 300
#PARSING STARTED#
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [6]:
df = pd.DataFrame(data)

import os
from datasets import Dataset, Features, ClassLabel, Image, Value, concatenate_datasets
from PIL import Image as PILImage

# Путь к основной папке с категориями
main_folder = "/content/images"

# Создаем структуру данных для хранения меток и изображений
data_h = {
    "images": [],
    "price" : df["price"].tolist()
}

# Проходим по каждой категории (например, cat, dog)
for category in os.listdir(main_folder):
    if category == ".ipynb_checkpoints":
      continue
    category_path = os.path.join(main_folder, category)

    if os.path.isdir(category_path):
        images = []
        # Собираем все изображения в этой категории и сохраняем их пути
        for img_file in os.listdir(category_path):
            if img_file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(category_path, img_file)
                images.append(img_path)  # Сохраняем путь к изображению

        # Добавляем в датасет: метку и список изображений
        data_h["images"].append(images)   # Список путей к изображениям

data_h["url"] = df.link.values.tolist()
# Создаем датасет с правильным форматом: метки и изображения
features = Features({
    "images": [Image()],  # Задаем тип данных для изображений как Image
    "url": Value("string"),
    "price" : Value("string")
})

# Преобразуем данные в датасет Hugging Face
new_dataset = Dataset.from_dict(data_h, features=features)

# Проверяем структуру датасета
print(new_dataset)

Dataset({
    features: ['images', 'price', 'url'],
    num_rows: 327
})


In [7]:
from datasets import concatenate_datasets, load_dataset
dataset = concatenate_datasets([load_dataset("I77/house_kg_flats")["train"], new_dataset])

print(dataset.to_pandas().url.unique().shape)


(1696,)


In [8]:
# Отправляем датасет на Hugging Face Hub
dataset.push_to_hub("I77/house_kg_flats")  #пусть к датасету, надо сначала создать в личном кабинете на huggingface

Uploading the dataset shards:   0%|          | 0/4 [00:00<?, ?it/s]

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Map:   0%|          | 0/424 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/I77/house_kg_flats/commit/d00aafc7839931e0df20921329cca979eb634bf8', commit_message='Upload dataset', commit_description='', oid='d00aafc7839931e0df20921329cca979eb634bf8', pr_url=None, pr_revision=None, pr_num=None)